# Create the evolution data ( NO PLOTTING)

For now, matplotlib.pyplot is in conflict with pandora.astro (which is needed to evolve galaxies). Therefore to run this notebook you want to use a python installed with conda from the package that Paolo Franzetti made. 

I am using:  /Users/giorgiomanzoni/PANDORA/envs/pandora.ez/bin/jupyter-notebook

In [1]:
from pandora.astro import Mags

In [2]:
from fase.fileio import Table

In [3]:
from fase.fileio import Image

In [4]:
from fase.fileio import Spectrum

In [6]:
from astropy.cosmology import FlatLambdaCDM

In [7]:
import numpy as np

In [193]:
from GMfunctions import *

In [9]:
import os

In [229]:
#del sys.modules['GMfunctions']
import string

# Function to read magnitudes from the evolved model

In [10]:
def getUV_NewModel(templ_start,delta_t):
    jump_n_model = int((delta_t/0.1)+0.5)
    templ_end = templ_start + jump_n_model

    #get tau of templ_start and compare it to the tau of templ_end

    if templ_end<=1 or ima.getRow(templ_end)[tao_position] != ima.getRow(templ_start)[tao_position]:
        ERROR= True
    else:
        ERROR=False

    if not ERROR:
        #print "numero di modelli da saltare:", jump_n_model, "--> delta t= ",delta_t
        flx_all = ima.getRow(templ_end)
        flx = flx_all[0:last_lamb+1]
        spec = Spectrum((lambdas,flx))
        u = spec.getMag("U_Bessel",1,mag)
        v = spec.getMag("V_Buser",1,mag)
        logmass = flx_all[mass_position]
    else:
        u = -99.
        v = -99.
        logmass = -99.
    return u,v,logmass,ERROR

# Read variables

In [231]:
match = Table("../data/Data_Sample_V7.fits",hdu=8)
# image containing PEGASE models for SFH with no quenching
ima = Image("../data/models_san_new_wind_abs.fits")
mag = Mags()

cosmo = FlatLambdaCDM(H0=70,Om0=0.3)

lambdas_all = ima.getRow(1)
last_lamb = lambdas_all.argmin()-1
lambdas = lambdas_all[0:last_lamb+1]

# position in pixels where the 
# parameters are stored in ima
tao_position  = last_lamb+1
age_position  = last_lamb+2
sfr_position  = last_lamb+6
mass_position = last_lamb+7
metallicity_position = last_lamb+5

In [12]:
z       = match.getCol("zspec_1")
templ   = match.getCol("TEMPL")
mag_u   = match.getCol("M_Uj")
mag_v   = match.getCol("M_V")
uv      = mag_u - mag_v
lgmass  = match.getCol("log_stellar_mass")

mtau = match.getCol("TAU_2")
mage = match.getCol("AGE_2")
mindex = match.getCol("num_1")

In [13]:
# zlow 	 zhigh 	 z_median 	 t_median 
# 4.000000000000000222e-01 5.000000000000000000e-01 4.659999907016754150e-01 8.668469387307764507e+00
# 5.000000000000000000e-01 5.999999999999999778e-01 5.529999732971191406e-01 8.056404053468638082e+00
# 5.999999999999999778e-01 6.999999999999999556e-01 6.470000147819519043e-01 7.465174918806797955e+00
# 6.999999999999999556e-01 8.000000000000000444e-01 7.491000294685363770e-01 6.894509193135578862e+00
# 8.000000000000000444e-01 9.000000000000000222e-01 8.486499786376953125e-01 6.400014398583118158e+00
# 9.000000000000000222e-01 1.000000000000000000e+00 9.408000111579895020e-01 5.989489134132708337e+00
# 1.000000000000000000e+00 1.300000000000000044e+00 1.090000033378601074e+00 5.406702162694932312e+00

# Computing median redshifts and cosmic time for evolution

In [15]:
zint_low  = [0.4,0.5,0.6,0.7,0.8,0.9,1.0]
zint_high = [0.5,0.6,0.7,0.8,0.9,1.0,1.1]
zetas = np.zeros(len(zint_low))
times = np.zeros(len(zint_low))
for i in range(len(zetas)):
    z_bin = np.where((z>zint_low[i])&(z<zint_high[i]))
    zetas[i] = np.median(z[z_bin])
    times[i] = cosmo.age(z=zetas[i]).value

In [16]:
times

array([ 8.66094395,  8.06143064,  7.4719106 ,  6.9014859 ,  6.40691613,
        6.00713169,  5.58117456])

# Create output folder - NO QUENCHING

In [23]:
# this is where to save output files
#nome= "/Users/giorgiomanzoni/Desktop/test/"
nome="../VIPERS/SYNTHETIC"
if not os.path.isdir(nome):
    os.mkdir(nome)
nome += "/NO_QUENCH/"
if not os.path.isdir(nome):
    os.mkdir(nome)
print "Created folder: "+nome

Created folder: ../VIPERS/SYNTHETIC/NO_QUENCH/


# Actual evolution of galaxies - NO QUENCHING

In [24]:
# LOOP Z BIN START
#for t_start_pos in reversed(range(len(zetas))):   #6 5 4 3 2 1 0
for t_start_pos in [6]:
    
    print "z start = "+str(zint_low[t_start_pos])+" < z < "+str(zint_high[t_start_pos])
    
    z_bin = np.where((z>zint_low[t_start_pos])&(z<zint_high[t_start_pos]))
    z1013 = z[z_bin]
    templ1013 = templ[z_bin]
    uv1013 = uv[z_bin]
    v1013 = mag_v[z_bin]
    mass1013 = lgmass[z_bin]
    tau1013 = mtau[z_bin]
    age1013 = mage[z_bin]
    index1013 = mindex[z_bin]

    for t_end_pos in reversed(range(t_start_pos)):# if BACK: (6)--> 5,4,3,2,1,0
        if t_end_pos == t_start_pos:
            continue

        DELTAT = times[t_end_pos]-times[t_start_pos]


        synthetic_v = []
        synthetic_uv = []
        synthetic_mass = []
        synthetic_tau = []
        synthetic_age = []
        # I know that does not change from the real but if
        # I want to use np.where I have to manain the array to be of the same dimension (the same for taus)
        synthetic_templ = []    
        synthetic_index = []
        
        
        
        for i in range(len(z1013)):
            ERROR = False
            fluxes_all = ima.getRow(templ1013[i]+1)
            fluxes = fluxes_all[0:last_lamb+1]
            spectra = Spectrum((lambdas,fluxes))

            U_i  = spectra.getMag("U_Bessel",1,mag)
            V_i  = spectra.getMag("V_Buser",1,mag)
            MASS_i = fluxes_all[mass_position]
            UV_initial = U_i - V_i

            tao = fluxes_all[tao_position]
            age = fluxes_all[age_position]

            new_age = age + DELTAT
            #_______________________________________________________________
            U_f ,V_f, MASS_f,ERROR = getUV_NewModel(templ1013[i]+1,DELTAT)
            #_______________________________________________________________
            
            if ERROR:
                raise Exception('Something went wrong while computing \
                            the evolution of the model'+str(templ1013[i]+"for "+str(DELTAT)+" Gyr"))
                
            UV_final  = U_f  - V_f
            delta_UV = UV_final - UV_initial
            delta_V  = V_f - V_i
            #delta_MASS = MASS_f - MASS_i
            delta_MASS_ratio = MASS_f / MASS_i

            if not ERROR:
                synthetic_v.append(v1013[i]+delta_V)
                synthetic_uv.append(uv1013[i]+delta_UV)
                synthetic_mass.append(mass1013[i]+np.log10(delta_MASS_ratio))
                synthetic_tau.append(tau1013[i])
                synthetic_age.append(age1013[i]+DELTAT)
                synthetic_templ.append(templ1013[i])
                synthetic_index.append(index1013[i])
            else:
                synthetic_v.append(-99.)
                synthetic_uv.append(-99.)
                synthetic_mass.append(-99.)
                synthetic_tau.append(-99.)
                synthetic_age.append(-99.)
                synthetic_templ.append(-99.)
                synthetic_index.append(-99.)

 
        synthetic_uv = np.array(synthetic_uv)
        synthetic_v = np.array(synthetic_v)
        synthetic_mass = np.array(synthetic_mass)
        synthetic_tau = np.array(synthetic_tau)
        synthetic_age = np.array(synthetic_age)
        synthetic_templ = np.array(synthetic_templ)
        synthetic_index = np.array(synthetic_index)
        
        #SAVE SYNTHETIC GALAXIES IN A TXT FILE
        SYN_POINTS_filename = nome+"SYNTHETIC_points_%i%i_%i%i.txt"%(zint_low[t_start_pos]*10,zint_high[t_start_pos]*10,zint_low[t_end_pos]*10,zint_high[t_end_pos]*10)
        data_syn = [synthetic_uv,synthetic_v,synthetic_mass]
        data_syn = np.array(data_syn)
        np.savetxt(SYN_POINTS_filename,data_syn.T,header="syn_color_uv \t syn_mag_v \t syn_log_mass")


z start = 1.0 < z < 1.1


In [154]:
# make sure you use random.seed(0) when running random QUENCH 

In [186]:
print np.random.seed(0)
print np.random.choice([0,1,2,3,4])

None
4


# QUENCHING

# Function to read from PEGASE txt files

In [262]:
def CheckWarning(file):
    WARNING=False
    f = open(file,"r")
    lines = f.readlines()
    f.close()
    # ignore header
    i=0
    while lines[i][0] != '*':
        if "WARNING" in lines[i]:
            WARNING = True
        i+=1
    return WARNING

In [263]:
def getQuenchedLAMBDAS(dir_models,weak=False):
    '''it returns the wavelenghts (common in every file of the project) and
    numbers of line for each element of PEGASE format (relative to a single spectrum)'''

    WARNING = False

    if not weak:
        os.system("ls "+ dir_models+" >> ./ls_quench.txt")
    else:
        os.system("ls "+ dir_models+" >> ./ls_quench_weak.txt")


    f = open("./ls_quench.txt","r")
    f.readline() # less dangerous the second line of the file (so this command skips the first)
    nome_file = f.readline()
    f.close()

    if "\n" in nome_file:
        nome_file = nome_file[:-1]

    f = open(dir_models+nome_file,"r")
    lines = f.readlines()
    f.close()

    # ignore header
    i=0
    while lines[i][0] != '*':
        if "WARNING" in lines[i]:
            WARNING = True
        i+=1
    #i is in the row of ****
    i +=1
    len_header = i
    tsteps, nwaves, nlines = map(int, string.split(lines[i]))

    #compute how many rows are dedicated to wavelenghts
    wavelines = float(nwaves)/5
    if int(wavelines) != wavelines:
        #add a line if number of wavelenghts are not multiple of 5
        #remember that int(1.9)==1
        wavelines += 1

    #number of rows for wavelenghts
    wavelines = int(wavelines)

    #seek the index to the first wavelenght
    i +=1

    #load wavelenghts
    waves=[]
    for line in lines[i:i+wavelines]:
        waves += map(float, string.split(line))
        i+=1

    #now i points to the first row of emission lines wavelenghts

    waves = np.array(waves,np.float64)
    #wave_cut = np.compress((waves>=MIN_WAVES)&(waves<=MAX_WAVES),waves)

    #compute how many rows are dedicated to emission lines
    lwavelines = float(nlines)/5
    if int(lwavelines)!= lwavelines:
        lwavelines += 1

    #number of rows for emission lines
    lwavelines = int(lwavelines)
    return waves,len_header, wavelines, lwavelines, WARNING

In [264]:
def getQuenchedUV(tau,quench,age,path=model_path):

    taustr = string.replace(string.replace('%5.2f' % tau, ' ', '0'),'.','-')
    quenchstr = string.replace(string.replace('%5.2f' % quench, ' ', '0'),'.','-')
        
    model_file = path+'salp_san_t%s_quench%s.spectra' % (taustr, quenchstr)


    f = open(model_file,"r")
    lines = f.readlines()
    f.close()

    if CheckWarning(model_file):
        i = ROW_HEADER + 1 + ROW_WAVES + ROW_EMIS_LINE + 1
    else:
        i = ROW_HEADER + ROW_WAVES + ROW_EMIS_LINE + 1

    #e.g. age=6.1 ==> you have to skip (61-1) blocks because age=0.1 is the zeroth block
    seek = int(age*10)-1
    i += seek * (ROW_WAVES+ROW_EMIS_LINE+2)
    
    #print lines[i]
    #print lines[i+1]
    #printRedBlink(model_file)

    param1 = map(float,string.split(lines[i]))
    i +=1
    #param2 = map(float,string.split(lines[i]))
    i +=1

    fluxes=[]
    for line in lines[i:i+ROW_WAVES]:
        fluxes += map(float, string.split(line))
        i+=1

    spec = Spectrum((lambdas,fluxes))
    u = spec.getMag("U_Bessel",1,mag)
    v = spec.getMag("V_Buser",1,mag)
    mass_norm = param1[2]


    return u,v, mass_norm

## Input parameters

In [276]:
limited= 3 # [0,1,2,3] #interval time in which the random quenching has to occur after the age of the galaxy
output_path = createPath("../VIPERS/SYNTHETIC/QUENCH_"+str(int(limited))+"/")
model_path = "/Volumes/SP PHD U3/GalaxyEvolution/Analisi_CMR/models/QUENCHED_MODELS/"

## Reading parameters in order to read from PEGASE txt files

In [277]:
lambdas,ROW_HEADER, ROW_WAVES, ROW_EMIS_LINE, WARNING = getQuenchedLAMBDAS(model_path)

In [278]:
if WARNING:
    ROW_HEADER = ROW_HEADER-1

In [279]:
# I am just changing the name of AGE and TAU in order to be more practical with the code

TAU = match.getCol("TAU_2")
AGE = match.getCol("AGE_2")

AGE_1 = np.unique(AGE)

# RUNNING THE EVOLUTION with quenching limited by the variable "limited"

In [280]:
#LOOP ON THE Z BIN TO START
#for t_start_pos in reversed(range(len(zetas))):   #6 5 4 3 2 1 0
for t_start_pos in [6]:
    
    z_bin = np.where((z>zint_low[t_start_pos])&(z<zint_high[t_start_pos]))
    z1013 = z[z_bin]
    templ1013 = templ[z_bin]
    uv1013 = uv[z_bin]
    v1013 = mag_v[z_bin]
    mass1013 = lgmass[z_bin]

    tau1013 = TAU[z_bin]
    age1013 = AGE[z_bin]
    
    #LOOP ON THE Z BIN TO END
    for t_end_pos in reversed(range(t_start_pos)):# if BACK: (6)--> 5,4,3,2,1,0
        if t_end_pos == t_start_pos:
            continue

        DELTAT = times[t_end_pos]-times[t_start_pos]

        synthetic_v = []
        synthetic_uv = []
        synthetic_mass = []

        # If I don't set the seed, every time I change end redshift bin, 
        # every galaxy follow a different quenching since it is random
        np.random.seed(0)
        for i in range(len(z1013)):
            ERROR = False

            #in genera_evoluzio_random I choose a different quench instead to fix it at the age of the galaxy:
            #in function getQuenchedUV I have to change the second parameter.
            #in the following code lines I choose it.


            if limited!=0:
                #possible_quench = np.where(AGE_1>age1013[i])#without limitations in time
                possible_quench3 = np.where((AGE_1>age1013[i])&(np.fabs(AGE_1-age1013[i])<float(limited)))
                #I require < 3 to make the quench fall in my observed evolution  (3.26 Gyr is the max delta_t)
                quenches = AGE_1[possible_quench3]
                try:
                    rand_quench = np.random.choice(quenches)
                except:
                    rand_quench = age1013[i]
            else:
                rand_quench = age1013[i]

            U_i,V_i,MASS_norm_i = getQuenchedUV(tau1013[i], rand_quench, age1013[i])

            UV_initial = U_i - V_i

            new_age = age1013[i]+ DELTAT


            #just to approximate at the first decimal figure
            #it should be equivalent to use round(new_age,1)
            new_age_approx = (int(new_age*10+0.5)*0.1)


            if new_age_approx<=0.0:
                ERROR = True
                printRedBlink("\nTOO MUCH BACK IN TIME!!!!!!!!!!\n")

            if not ERROR:
                U_f, V_f, MASS_norm_f = getQuenchedUV(tau1013[i],rand_quench,new_age_approx)
            else:
                U_f = -99.
                V_f = -99.
                MASS_norm_f = -99.

            UV_final  = U_f  - V_f


            delta_UV = UV_final - UV_initial
            delta_V  = V_f - V_i
            #delta_MASS = MASS_f - MASS_i
            delta_MASS_ratio = MASS_norm_f/MASS_norm_i

            if not ERROR:
                synthetic_v.append(v1013[i]+delta_V)
                synthetic_uv.append(uv1013[i]+delta_UV)
                synthetic_mass.append(mass1013[i]+np.log10(delta_MASS_ratio))
            else:
                synthetic_v.append(-99.)
                synthetic_uv.append(-99.)
                synthetic_mass.append(-99.)

                
        synthetic_uv = np.array(synthetic_uv)
        synthetic_v = np.array(synthetic_v)
        synthetic_mass = np.array(synthetic_mass)

        
        #SAVE SYNTHETIC GALAXIES IN A TXT FILE
        SYN_POINTS_filename = output_path+"SYNTHETIC_points_%i%i_%i%i.txt"%(zint_low[t_start_pos]*10,zint_high[t_start_pos]*10,zint_low[t_end_pos]*10,zint_high[t_end_pos]*10)
        data_syn = [synthetic_uv,synthetic_v,synthetic_mass]
        data_syn = np.array(data_syn)
        np.savetxt(SYN_POINTS_filename,data_syn.T,header="syn_color_uv \t syn_mag_v \t syn_log_mass")
    

In [ ]:
variable = 12.345

In [294]:
string.replace("ciao%8.2fciao"%1.23234," ","$")

'ciao$$$$1.23ciao'

In [289]:
type(0.1)

float

In [291]:
type(limited)

int

In [298]:
string.replace(string.replace('%5.2f' % 2.5, ' ', ''),'.','_')

'2_50'